In [1]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs # library for plotting on maps
import cartopy.feature as cfeature # library for plotting states and boundaries
import matplotlib.pyplot as plt  # library for plotting
import os
import glob
import pandas as pd
import matplotlib.animation as animation
import datetime as dt # library for working with years, dates, etc.
from datetime import datetime, timedelta
from scipy.stats import ks_2samp

------------------------------------------------------------------------------------------------
Changes in Precipitation Type Occurrences During 5000 Grid Point Dual Frozen Precipitation Times
------------------------------------------------------------------------------------------------

In [2]:
### Read in the precipitation data
HistoricalPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_Hist_Precip_Accumulations.nc', engine = 'netcdf4')
EndPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_End_Precip_Accumulations.nc', engine = 'netcdf4')
HistoricalPrecipData

<xarray.Dataset> Size: 87GB
Dimensions:         (Time: 21847, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 175kB 1990-10-01 ... 2005-09-30
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 17GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 17GB ...
    Rain_Accum      (Time, south_north, west_east) float32 17GB ...
    Ice_Accum       (Time, south_north, west_east) float32 17GB ...
    Fzra_Accum      (Time, south_north, west_east) float32 17GB ...

In [3]:
### Find times in the data where there is freezing precipitation of at least two types greater than a certain value
## Historical
# Create masks for freezing precipitation types above the threshold
H_fzra_mask = HistoricalPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_ice_mask = HistoricalPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_snow_mask = HistoricalPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
H_fzra_count = H_fzra_mask.sum(dim=["south_north", "west_east"])
H_ice_count = H_ice_mask.sum(dim=["south_north", "west_east"])
H_snow_count = H_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the space threshold at each time step
H_freezing_precip5000_count = ((H_fzra_count >= 5000).astype(int) + (H_ice_count >= 5000).astype(int) + (H_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their space thresholds
H_valid_times = HistoricalPrecipData["Time"].where(H_freezing_precip5000_count >= 2, drop=True)
print('Number of Historical Times With at Least Two Types of Frozen Precipitation:', len(H_valid_times['Time']))

## End of Century
# Create masks for freezing precipitation types above the threshold
E_fzra_mask = EndPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_ice_mask = EndPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_snow_mask = EndPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
E_fzra_count = E_fzra_mask.sum(dim=["south_north", "west_east"])
E_ice_count = E_ice_mask.sum(dim=["south_north", "west_east"])
E_snow_count = E_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the threshold at each time step
E_freezing_precip5000_count = ((E_fzra_count >= 5000).astype(int) + (E_ice_count >= 5000).astype(int) + (E_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their thresholds
E_valid_times = EndPrecipData["Time"].where(E_freezing_precip5000_count >= 2, drop=True)
print('Number of End of Century Times With at Least Two Types of Frozen Precipitation:', len(E_valid_times['Time']))

Number of Historical Times With at Least Two Types of Frozen Precipitation: 3111
Number of End of Century Times With at Least Two Types of Frozen Precipitation: 2110


In [4]:
### Subset the data for only the times with at least two types of frozen precipitation that span over 5000 grid points each
H_precip_type_5000_data = HistoricalPrecipData.sel(Time=H_valid_times)
E_precip_type_5000_data = EndPrecipData.sel(Time=E_valid_times)
H_precip_type_5000_data

<xarray.Dataset> Size: 12GB
Dimensions:         (Time: 3111, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 25kB 1990-11-05T12:00:00 ... 2005-0...
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 2GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Rain_Accum      (Time, south_north, west_east) float32 2GB ...
    Ice_Accum       (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Fzra_Accum      (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0

In [5]:
### Calculate the number of occurrences of each precipitation type at each grid point throughout both of the periods over times that have
### two types of frozen precipitation that span over 5000 grid points each
## Historical
# Apply the 6 hourly precip thresholds to filter the precipitation type data for the historical period
H_fzra_5000_filtered = H_precip_type_5000_data['Fzra_Accum'].where(H_precip_type_5000_data['Fzra_Accum'] >= 0.254, drop=False) # Greater than 0.01 inches or 0.254 mm
H_ice_5000_filtered = H_precip_type_5000_data['Ice_Accum'].where(H_precip_type_5000_data['Ice_Accum'] >= 0.254, drop=False) # Greater than 0.01 inches or 0.254 mm
H_snowfall_5000_filtered = H_precip_type_5000_data['Snowfall_Accum'].where(H_precip_type_5000_data['Snowfall_Accum'] >= 2.54, drop=False) # Greater than 0.1 inches or 2.54 mm
H_rainfall_5000_filtered = H_precip_type_5000_data['Rain_Accum'].where(H_precip_type_5000_data['Rain_Accum'] >= 2.54, drop=False) # Greater than 0.1 inches or 2.54 mm

## End of Century
# Apply the 6 hourly precip thresholds to filter the precipitation type data for the end of century period
E_fzra_5000_filtered = E_precip_type_5000_data['Fzra_Accum'].where(E_precip_type_5000_data['Fzra_Accum'] >= 0.254, drop=False) # Greater than 0.01 inches or 0.254 mm
E_ice_5000_filtered = E_precip_type_5000_data['Ice_Accum'].where(E_precip_type_5000_data['Ice_Accum'] >= 0.254, drop=False) # Greater than 0.01 inches or 0.254 mm
E_snowfall_5000_filtered = E_precip_type_5000_data['Snowfall_Accum'].where(E_precip_type_5000_data['Snowfall_Accum'] >= 2.54, drop=False) # Greater than 0.1 inches or 2.54 mm
E_rainfall_5000_filtered = E_precip_type_5000_data['Rain_Accum'].where(E_precip_type_5000_data['Rain_Accum'] >= 2.54, drop=False) # Greater than 0.1 inches or 2.54 mm

# Count the number of occurrences at each grid point for the historical period
H_snowfall_5000_count_spatial = H_snowfall_5000_filtered.count(dim="Time")
H_rainfall_5000_count_spatial = H_rainfall_5000_filtered.count(dim="Time")
H_ice_5000_count_spatial = H_ice_5000_filtered.count(dim="Time")
H_fzra_5000_count_spatial = H_fzra_5000_filtered.count(dim="Time")

# Count the number of occurrences at each grid point for the end-of-century period
E_snowfall_5000_count_spatial = E_snowfall_5000_filtered.count(dim="Time")
E_rainfall_5000_count_spatial = E_rainfall_5000_filtered.count(dim="Time")
E_ice_5000_count_spatial = E_ice_5000_filtered.count(dim="Time")
E_fzra_5000_count_spatial = E_fzra_5000_filtered.count(dim="Time")

# Take the difference between counts at each grid point to see the change
snowfall_5000_count_spatial_change = E_snowfall_5000_count_spatial - H_snowfall_5000_count_spatial
rainfall_5000_count_spatial_change = E_rainfall_5000_count_spatial - H_rainfall_5000_count_spatial
ice_5000_count_spatial_change = E_ice_5000_count_spatial - H_ice_5000_count_spatial
fzra_5000_count_spatial_change = E_fzra_5000_count_spatial - H_fzra_5000_count_spatial

In [7]:
### Create a 4 panel plot of the the differences in the occurrences of each precipitation type between periods over times that have two types of 
### frozen precipitation that span over 5000 grid points each
fig = plt.figure(figsize=(10, 12))

projection = ccrs.PlateCarree()

# Define axes manually
ax1 = fig.add_axes([0.08, 0.66, 0.38, 0.35], projection=projection)  # Top-left
ax2 = fig.add_axes([0.55, 0.66, 0.38, 0.35], projection=projection)  # Top-right
ax3 = fig.add_axes([0.08, 0.45, 0.38, 0.35], projection=projection)  # Bottom-left
ax4 = fig.add_axes([0.55, 0.45, 0.38, 0.35], projection=projection)  # Bottom-right

######## Snowfall Change in occurrence
# Add features
ax1.add_feature(cfeature.COASTLINE)
ax1.add_feature(cfeature.BORDERS)
ax1.add_feature(cfeature.STATES)
gl = ax1.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the snowfall occurrence change
snowfall_count_change_spatial = ax1.pcolormesh(snowfall_5000_count_spatial_change['lon'], snowfall_5000_count_spatial_change['lat'], 
                                            snowfall_5000_count_spatial_change, transform=projection, cmap='coolwarm', vmin=-500, vmax=500)

fig.colorbar(snowfall_count_change_spatial, ax=ax1, label='Change in #\nof Occurrences', shrink=0.4)
ax1.set_title('Change in Snowfall Occurrences', fontsize=10)

######## Rain Change in occurrence
# Add features
ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.BORDERS)
ax2.add_feature(cfeature.STATES)
gl = ax2.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the rainfall occurrence change
rainfall_count_change_spatial = ax2.pcolormesh(rainfall_5000_count_spatial_change['lon'], rainfall_5000_count_spatial_change['lat'], 
                                               rainfall_5000_count_spatial_change, transform=projection, cmap='coolwarm', vmin=-200, vmax=200)

fig.colorbar(rainfall_count_change_spatial, ax=ax2, label='Change in #\nof Occurrences', shrink=0.4)
ax2.set_title('Change in Rainfall Occurrences', fontsize=10)

######## Ice Change in occurrence
# Add features
ax3.add_feature(cfeature.COASTLINE)
ax3.add_feature(cfeature.BORDERS)
ax3.add_feature(cfeature.STATES)
gl = ax3.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the ice occurrence change
ice_count_change_spatial = ax3.pcolormesh(ice_5000_count_spatial_change['lon'], ice_5000_count_spatial_change['lat'], 
                                          ice_5000_count_spatial_change, transform=projection, cmap='coolwarm', vmin=-75, vmax=75)

fig.colorbar(ice_count_change_spatial, ax=ax3, label='Change in #\nof Occurrences', shrink=0.4)
ax3.set_title('Change in Ice Occurrences', fontsize=10)

######## Freezing Rain Change in occurrence
# Add features
ax4.add_feature(cfeature.COASTLINE)
ax4.add_feature(cfeature.BORDERS)
ax4.add_feature(cfeature.STATES)
gl = ax4.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the freezing rain occurrence change
fzra_count_change_spatial = ax4.pcolormesh(fzra_5000_count_spatial_change['lon'], fzra_5000_count_spatial_change['lat'], 
                                           fzra_5000_count_spatial_change, transform=projection, cmap='coolwarm', vmin=-150, vmax=150)

fig.colorbar(fzra_count_change_spatial, ax=ax4, label='Change in #\nof Occurrences', shrink=0.4)
ax4.set_title('Change in Freezing Rain Occurrences', fontsize=10)

# Add panel labels
ax1.text(0.01, 0.98, '(a)', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax2.text(0.01, 0.98, '(b)', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax3.text(0.01, 0.98, '(c)', transform=ax3.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax4.text(0.01, 0.98, '(d)', transform=ax4.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

plt.suptitle('Changes in Precipitation Type Occurrences During Times With\nAt Least Two Types of Frozen Precipitation Spanning 5000 Grid Points\nEnd of Century - Historical')

plt.savefig('Key Figures/Changes in Precipitation Type Occurrences During 5000 Grid Point Dual Precipitation Times',bbox_inches = 'tight')

plt.close()

----------------------------------------------------------------------------------------------
Changes in Precipitation Type Intensity During 5000 Grid Point Dual Frozen Precipitation Times
----------------------------------------------------------------------------------------------

In [2]:
### Read in the precipitation data
HistoricalPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_Hist_Precip_Accumulations.nc', engine = 'netcdf4')
EndPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_End_Precip_Accumulations.nc', engine = 'netcdf4')

In [3]:
### Find a times in the data where there is freezing precipitation of at least two types greater than a certain value
## Historical
# Create masks for freezing precipitation types above the threshold
H_fzra_mask = HistoricalPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_ice_mask = HistoricalPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_snow_mask = HistoricalPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
H_fzra_count = H_fzra_mask.sum(dim=["south_north", "west_east"])
H_ice_count = H_ice_mask.sum(dim=["south_north", "west_east"])
H_snow_count = H_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the space threshold at each time step
H_freezing_precip5000_count = ((H_fzra_count >= 5000).astype(int) + (H_ice_count >= 5000).astype(int) + (H_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their space thresholds
H_valid_times = HistoricalPrecipData["Time"].where(H_freezing_precip5000_count >= 2, drop=True)
print('Number of Historical Times With at Least Two Types of Frozen Precipitation:', len(H_valid_times['Time']))

## End of Century
# Create masks for freezing precipitation types above the threshold
E_fzra_mask = EndPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_ice_mask = EndPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_snow_mask = EndPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
E_fzra_count = E_fzra_mask.sum(dim=["south_north", "west_east"])
E_ice_count = E_ice_mask.sum(dim=["south_north", "west_east"])
E_snow_count = E_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the threshold at each time step
E_freezing_precip5000_count = ((E_fzra_count >= 5000).astype(int) + (E_ice_count >= 5000).astype(int) + (E_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their thresholds
E_valid_times = EndPrecipData["Time"].where(E_freezing_precip5000_count >= 2, drop=True)
print('Number of End of Century Times With at Least Two Types of Frozen Precipitation:', len(E_valid_times['Time']))

Number of Historical Times With at Least Two Types of Frozen Precipitation: 3111
Number of End of Century Times With at Least Two Types of Frozen Precipitation: 2110


In [4]:
### Subset the data for only the times with at least two types of frozen precipitation that span over 5000 grid points each
H_precip_type_5000_data = HistoricalPrecipData.sel(Time=H_valid_times)
E_precip_type_5000_data = EndPrecipData.sel(Time=E_valid_times)
H_precip_type_5000_data

<xarray.Dataset> Size: 12GB
Dimensions:         (Time: 3111, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 25kB 1990-11-05T12:00:00 ... 2005-0...
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 2GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Rain_Accum      (Time, south_north, west_east) float32 2GB ...
    Ice_Accum       (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Fzra_Accum      (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0

In [5]:
### Find the average 6-hourly precipitation type accumulation at each point in the domain over the selected dual frozen precipitation times
# Define the precipitation type intensity averages during the historical selected times
H_precip_type_5000_snowfallavg = H_precip_type_5000_data['Snowfall_Accum'].mean(dim='Time')
H_precip_type_5000_rainavg = H_precip_type_5000_data['Rain_Accum'].mean(dim='Time')
H_precip_type_5000_iceavg = H_precip_type_5000_data['Ice_Accum'].mean(dim='Time')
H_precip_type_5000_fzraavg = H_precip_type_5000_data['Fzra_Accum'].mean(dim='Time')

# Define the precipitation type intensity averages during the end of century selected times
E_precip_type_5000_snowfallavg = E_precip_type_5000_data['Snowfall_Accum'].mean(dim='Time')
E_precip_type_5000_rainavg = E_precip_type_5000_data['Rain_Accum'].mean(dim='Time')
E_precip_type_5000_iceavg = E_precip_type_5000_data['Ice_Accum'].mean(dim='Time')
E_precip_type_5000_fzraavg = E_precip_type_5000_data['Fzra_Accum'].mean(dim='Time')

In [6]:
### Calculate the average 6-hourly precipitation type intensity change at each point in the domain over the selected dual frozen precipitation times
snowfallavg_change_5000 = E_precip_type_5000_snowfallavg - H_precip_type_5000_snowfallavg
rainavg_change_5000 = E_precip_type_5000_rainavg - H_precip_type_5000_rainavg
iceavg_change_5000 = E_precip_type_5000_iceavg - H_precip_type_5000_iceavg
fzraavg_change_5000 = E_precip_type_5000_fzraavg - H_precip_type_5000_fzraavg

In [10]:
### Read in the bootstrapped p-values for each precipitation type and rename the variable
Snowfall_1000iteration_pvals = xr.open_dataset('Bootstrapping P-Values/Snowfall_5000points_Bootstrapping1000_Pvals.nc', engine = 'netcdf4')
Fzra_1000iteration_pvals = xr.open_dataset('Bootstrapping P-Values/Freezing_Rain_5000points_Bootstrapping1000_Pvals.nc', engine = 'netcdf4')
Ice_1000iteration_pvals = xr.open_dataset('Bootstrapping P-Values/Ice_5000points_Bootstrapping1000_Pvals.nc', engine = 'netcdf4')
Rain_1000iteration_pvals = xr.open_dataset('Bootstrapping P-Values/Rain_5000points_Bootstrapping1000_Pvals.nc', engine = 'netcdf4')

# Rename the p-value variable to be called
Snowfall_1000iteration_pvals = Snowfall_1000iteration_pvals.rename({'__xarray_dataarray_variable__': 'P-Values'})
Fzra_1000iteration_pvals = Fzra_1000iteration_pvals.rename({'__xarray_dataarray_variable__': 'P-Values'})
Ice_1000iteration_pvals = Ice_1000iteration_pvals.rename({'__xarray_dataarray_variable__': 'P-Values'})
Rain_1000iteration_pvals = Rain_1000iteration_pvals.rename({'__xarray_dataarray_variable__': 'P-Values'})

In [26]:
### Create a 4 panel plot of the average changes in each precipitation type intensity across the domain with bootstrapping 
### showing statistical significance
fig = plt.figure(figsize=(10, 12))

projection = ccrs.PlateCarree()

# Define axes manually
ax1 = fig.add_axes([0.08, 0.66, 0.38, 0.35], projection=projection)  # Top-left
ax2 = fig.add_axes([0.55, 0.66, 0.38, 0.35], projection=projection)  # Top-right
ax3 = fig.add_axes([0.08, 0.45, 0.38, 0.35], projection=projection)  # Bottom-left
ax4 = fig.add_axes([0.55, 0.45, 0.38, 0.35], projection=projection)  # Bottom-right

######## Snowfall Change
# Map the significance at the 99% confidence level
alpha = 0.01
snowfall_hatching = ((Snowfall_1000iteration_pvals['P-Values']<=alpha/2) | (Snowfall_1000iteration_pvals['P-Values']>=1-alpha/2)) # Above or below alpha divided by 2 to capture both tailed extremes

# Add features
ax1.add_feature(cfeature.COASTLINE)
ax1.add_feature(cfeature.BORDERS)
ax1.add_feature(cfeature.STATES)
gl = ax1.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average snowfall change
snowfallavg_change_spatial = ax1.pcolormesh(snowfallavg_change_5000['lon'], snowfallavg_change_5000['lat'], snowfallavg_change_5000, transform=projection,
                                cmap='coolwarm', vmin=-7, vmax=7)

# Overlay the significant points with hatching from the bootstrapping
ax1.contourf(snowfallavg_change_5000['lon'], snowfallavg_change_5000['lat'], snowfall_hatching, 
             levels=[0.5, 1], colors='none', hatches=['////'], transform=projection)

fig.colorbar(snowfallavg_change_spatial, ax=ax1, label='Change in 6-Hourly\nAverage (mm)', shrink=0.4)
ax1.set_title('Change in Average Snowfall Intensity', fontsize=10)

######## Rain Change
# Map the significance at the 99% confidence level
alpha = 0.01
rain_hatching = ((Rain_1000iteration_pvals['P-Values']<=alpha/2) | (Rain_1000iteration_pvals['P-Values']>=1-alpha/2)) # Above or below alpha divided by 2 to capture both tailed extremes

# Add features
ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.BORDERS)
ax2.add_feature(cfeature.STATES)
gl = ax2.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average ice change
rainavg_change_spatial = ax2.pcolormesh(rainavg_change_5000['lon'], rainavg_change_5000['lat'], rainavg_change_5000, transform=projection,
                                cmap='coolwarm', vmin=-0.8, vmax=0.8)

# Overlay the significant points with hatching from the bootstrapping
ax2.contourf(rainavg_change_5000['lon'], rainavg_change_5000['lat'], rain_hatching, 
             levels=[0.5, 1], colors='none', hatches=['////'], transform=projection)

fig.colorbar(rainavg_change_spatial, ax=ax2, label='Change in 6-Hourly\nAverage (mm)', shrink=0.4)
ax2.set_title('Change in Average Rainfall Intensity', fontsize=10)

######## Ice Change
# Map the significance at the 99% confidence level
alpha = 0.01
ice_hatching = ((Ice_1000iteration_pvals['P-Values']<=alpha/2) | (Ice_1000iteration_pvals['P-Values']>=1-alpha/2)) # Above or below alpha divided by 2 to capture both tailed extremes

# Add features
ax3.add_feature(cfeature.COASTLINE)
ax3.add_feature(cfeature.BORDERS)
ax3.add_feature(cfeature.STATES)
gl = ax3.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average rainfall change
iceavg_change_spatial = ax3.pcolormesh(iceavg_change_5000['lon'], iceavg_change_5000['lat'], iceavg_change_5000, transform=projection,
                                cmap='coolwarm', vmin=-0.1, vmax=0.1)

# Overlay the significant points with hatching from the bootstrapping
ax3.contourf(iceavg_change_5000['lon'], iceavg_change_5000['lat'], ice_hatching, 
             levels=[0.5, 1], colors='none', hatches=['////'], transform=projection)

fig.colorbar(iceavg_change_spatial, ax=ax3, label='Change in 6-Hourly\nAverage (mm)', shrink=0.4)
ax3.set_title('Change in Average Ice Intensity', fontsize=10)

######## Freezing Rain Change
# Map the significance at the 99% confidence level
alpha = 0.01
fzra_hatching = ((Fzra_1000iteration_pvals['P-Values']<=alpha/2) | (Fzra_1000iteration_pvals['P-Values']>=1-alpha/2)) # Above or below alpha divided by 2 to capture both tailed extremes

# Add features
ax4.add_feature(cfeature.COASTLINE)
ax4.add_feature(cfeature.BORDERS)
ax4.add_feature(cfeature.STATES)
gl = ax4.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average freezing rain change
fzraavg_change_spatial = ax4.pcolormesh(fzraavg_change_5000['lon'], fzraavg_change_5000['lat'], fzraavg_change_5000, transform=projection,
                                cmap='coolwarm', vmin=-0.3, vmax=0.3)

# Overlay the significant points with hatching from the bootstrapping
ax4.contourf(fzraavg_change_5000['lon'], fzraavg_change_5000['lat'], fzra_hatching, 
             levels=[0.5, 1], colors='none', hatches=['////'], transform=projection)

fig.colorbar(fzraavg_change_spatial, ax=ax4, label='Change in 6-Hourly\nAverage (mm)', shrink=0.4)
ax4.set_title('Change in Average Freezing Rain Intensity', fontsize=10)

# Add panel labels
ax1.text(0.01, 0.98, '(a)', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax2.text(0.01, 0.98, '(b)', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax3.text(0.01, 0.98, '(c)', transform=ax3.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax4.text(0.01, 0.98, '(d)', transform=ax4.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

plt.suptitle('Changes in Precipitation Type Intensity During Times With\nAt Least Two Types of Frozen Precipitation Spanning 5000 Grid Points\nEnd of Century - Historical')

plt.savefig('Key Figures/Changes in Precipitation Type Intensity During 5000 Grid Point Dual Precipitation Times',bbox_inches = 'tight')

plt.close()

-----------------------------------------------------------------------------------------------------------------------------
Changes in the Distribution of Each Precipitation Type Between Periods During 5000 Grid Point Dual Frozen Precipitation Times
-----------------------------------------------------------------------------------------------------------------------------

In [2]:
### Read in the precipitation data
HistoricalPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_Hist_Precip_Accumulations.nc', engine = 'netcdf4')
EndPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_End_Precip_Accumulations.nc', engine = 'netcdf4')
HistoricalPrecipData

<xarray.Dataset> Size: 87GB
Dimensions:         (Time: 21847, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 175kB 1990-10-01 ... 2005-09-30
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 17GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 17GB ...
    Rain_Accum      (Time, south_north, west_east) float32 17GB ...
    Ice_Accum       (Time, south_north, west_east) float32 17GB ...
    Fzra_Accum      (Time, south_north, west_east) float32 17GB ...

In [3]:
### Find times in the data where there is freezing precipitation of at least two types greater than a certain value
## Historical
# Create masks for freezing precipitation types above the threshold
H_fzra_mask = HistoricalPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_ice_mask = HistoricalPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_snow_mask = HistoricalPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
H_fzra_count = H_fzra_mask.sum(dim=["south_north", "west_east"])
H_ice_count = H_ice_mask.sum(dim=["south_north", "west_east"])
H_snow_count = H_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the space threshold at each time step
H_freezing_precip5000_count = ((H_fzra_count >= 5000).astype(int) + (H_ice_count >= 5000).astype(int) + (H_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their space thresholds
H_valid_times = HistoricalPrecipData["Time"].where(H_freezing_precip5000_count >= 2, drop=True)
print('Number of Historical Times With at Least Two Types of Frozen Precipitation:', len(H_valid_times['Time']))

## End of Century
# Create masks for freezing precipitation types above the threshold
E_fzra_mask = EndPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_ice_mask = EndPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_snow_mask = EndPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
E_fzra_count = E_fzra_mask.sum(dim=["south_north", "west_east"])
E_ice_count = E_ice_mask.sum(dim=["south_north", "west_east"])
E_snow_count = E_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the threshold at each time step
E_freezing_precip5000_count = ((E_fzra_count >= 5000).astype(int) + (E_ice_count >= 5000).astype(int) + (E_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their thresholds
E_valid_times = EndPrecipData["Time"].where(E_freezing_precip5000_count >= 2, drop=True)
print('Number of End of Century Times With at Least Two Types of Frozen Precipitation:', len(E_valid_times['Time']))

Number of Historical Times With at Least Two Types of Frozen Precipitation: 3111
Number of End of Century Times With at Least Two Types of Frozen Precipitation: 2110


In [4]:
### Subset the data for only the times with at least two types of frozen precipitation that span over 5000 grid points each
H_precip_type_5000_data = HistoricalPrecipData.sel(Time=H_valid_times)
E_precip_type_5000_data = EndPrecipData.sel(Time=E_valid_times)
H_precip_type_5000_data

<xarray.Dataset> Size: 12GB
Dimensions:         (Time: 3111, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 25kB 1990-11-05T12:00:00 ... 2005-0...
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 2GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Rain_Accum      (Time, south_north, west_east) float32 2GB ...
    Ice_Accum       (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Fzra_Accum      (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0

In [5]:
### Subset into each precipitation type for each period's times that have at least two types of frozen precipitation that span over 5000 grid points each
# Historical variables
Hist_5000_6hourly_Snowfall = H_precip_type_5000_data['Snowfall_Accum']
Hist_5000_6hourly_Rain = H_precip_type_5000_data['Rain_Accum']
Hist_5000_6hourly_Ice = H_precip_type_5000_data['Ice_Accum']
Hist_5000_6hourly_Fzra = H_precip_type_5000_data['Fzra_Accum']

# End of century variables
End_5000_6hourly_Snowfall = E_precip_type_5000_data['Snowfall_Accum']
End_5000_6hourly_Rain = E_precip_type_5000_data['Rain_Accum']
End_5000_6hourly_Ice = E_precip_type_5000_data['Ice_Accum']
End_5000_6hourly_Fzra = E_precip_type_5000_data['Fzra_Accum']

In [6]:
### Remove accumulation values less then certain thresholds to avoid zeros in the distributions and to mask out small accumulations
# Historical
Hist_5000_6hourly_Snowfall_filtered = Hist_5000_6hourly_Snowfall.where(Hist_5000_6hourly_Snowfall >= 2.54) # Greater than 0.1 inches or 2.54 mm
Hist_5000_6hourly_Rain_filtered = Hist_5000_6hourly_Rain.where(Hist_5000_6hourly_Rain >= 2.54) # Greater than 0.1 inches or 2.54 mm
Hist_5000_6hourly_Ice_filtered = Hist_5000_6hourly_Ice.where(Hist_5000_6hourly_Ice >= 0.254) # Greater than 0.01 inches or 0.254 mm
Hist_5000_6hourly_Fzra_filtered = Hist_5000_6hourly_Fzra.where(Hist_5000_6hourly_Fzra >= 0.254) # Greater than 0.01 inches or 0.254 mm

# End of century
End_5000_6hourly_Snowfall_filtered = End_5000_6hourly_Snowfall.where(End_5000_6hourly_Snowfall >= 2.54) # Greater than 0.1 inches or 2.54 mm
End_5000_6hourly_Rain_filtered = End_5000_6hourly_Rain.where(End_5000_6hourly_Rain >= 2.54) # Greater than 0.1 inches or 2.54 mm
End_5000_6hourly_Ice_filtered = End_5000_6hourly_Ice.where(End_5000_6hourly_Ice >= 0.254) # Greater than 0.01 inches or 0.254 mm
End_5000_6hourly_Fzra_filtered = End_5000_6hourly_Fzra.where(End_5000_6hourly_Fzra >= 0.254) # Greater than 0.01 inches or 0.254 mm

In [7]:
### Calculate the mean and variance of each precip type to estimate the parameters of the gamma distribution for each precip type accumulation
### during times that have at least two types of frozen precipitation that span over 5000 grid points in each period
## Historical
Hist_5000_6hourly_Snowfall_mean = Hist_5000_6hourly_Snowfall_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
Hist_5000_6hourly_Snowfall_var = Hist_5000_6hourly_Snowfall_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point
Hist_5000_6hourly_Rain_mean = Hist_5000_6hourly_Rain_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
Hist_5000_6hourly_Rain_var = Hist_5000_6hourly_Rain_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point
Hist_5000_6hourly_Ice_mean = Hist_5000_6hourly_Ice_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
Hist_5000_6hourly_Ice_var = Hist_5000_6hourly_Ice_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point
Hist_5000_6hourly_Fzra_mean = Hist_5000_6hourly_Fzra_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
Hist_5000_6hourly_Fzra_var = Hist_5000_6hourly_Fzra_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point

## End of Century
End_5000_6hourly_Snowfall_mean = End_5000_6hourly_Snowfall_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
End_5000_6hourly_Snowfall_var = End_5000_6hourly_Snowfall_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point
End_5000_6hourly_Rain_mean = End_5000_6hourly_Rain_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
End_5000_6hourly_Rain_var = End_5000_6hourly_Rain_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point
End_5000_6hourly_Ice_mean = End_5000_6hourly_Ice_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
End_5000_6hourly_Ice_var = End_5000_6hourly_Ice_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point
End_5000_6hourly_Fzra_mean = End_5000_6hourly_Fzra_filtered.mean(dim='Time', skipna=True) # Calculate the mean accumulation at each point
End_5000_6hourly_Fzra_var = End_5000_6hourly_Fzra_filtered.var(dim='Time', skipna=True) # Calculate the standard deviation of accumulation at each point

In [8]:
### Calculate the gamma parameters using methods of moments for each precipitation type during times that have at least two types 
### of frozen precipitation that span over 5000 grid points and in each period
# Historical
Hist_5000_Snowfall_gamma_shape = (Hist_5000_6hourly_Snowfall_mean ** 2) / Hist_5000_6hourly_Snowfall_var 
Hist_5000_Snowfall_gamma_scale = Hist_5000_6hourly_Snowfall_var / Hist_5000_6hourly_Snowfall_mean
Hist_5000_Rain_gamma_shape = (Hist_5000_6hourly_Rain_mean ** 2) / Hist_5000_6hourly_Rain_var 
Hist_5000_Rain_gamma_scale = Hist_5000_6hourly_Rain_var / Hist_5000_6hourly_Rain_mean
Hist_5000_Ice_gamma_shape = (Hist_5000_6hourly_Ice_mean ** 2) / Hist_5000_6hourly_Ice_var 
Hist_5000_Ice_gamma_scale = Hist_5000_6hourly_Ice_var / Hist_5000_6hourly_Ice_mean
Hist_5000_Fzra_gamma_shape = (Hist_5000_6hourly_Fzra_mean ** 2) / Hist_5000_6hourly_Fzra_var 
Hist_5000_Fzra_gamma_scale = Hist_5000_6hourly_Fzra_var / Hist_5000_6hourly_Fzra_mean

# End of Century
End_5000_Snowfall_gamma_shape = (End_5000_6hourly_Snowfall_mean ** 2) / End_5000_6hourly_Snowfall_var 
End_5000_Snowfall_gamma_scale = End_5000_6hourly_Snowfall_var / End_5000_6hourly_Snowfall_mean
End_5000_Rain_gamma_shape = (End_5000_6hourly_Rain_mean ** 2) / End_5000_6hourly_Rain_var 
End_5000_Rain_gamma_scale = End_5000_6hourly_Rain_var / End_5000_6hourly_Rain_mean
End_5000_Ice_gamma_shape = (End_5000_6hourly_Ice_mean ** 2) / End_5000_6hourly_Ice_var 
End_5000_Ice_gamma_scale = End_5000_6hourly_Ice_var / End_5000_6hourly_Ice_mean
End_5000_Fzra_gamma_shape = (End_5000_6hourly_Fzra_mean ** 2) / End_5000_6hourly_Fzra_var 
End_5000_Fzra_gamma_scale = End_5000_6hourly_Fzra_var / End_5000_6hourly_Fzra_mean

In [9]:
### Define a function to be used to sample for the precipitation accumulation distributions and also test for significant changes between the two distributions
def sample_and_test(hist_gamma_shape, hist_gamma_scale, end_gamma_shape, end_gamma_scale, n_samples=1000):
    from scipy.stats import gamma, ks_2samp
    # Skip invalid points
    if np.isnan(hist_gamma_shape) or np.isnan(end_gamma_shape):
        return np.nan
    # Generate samples
    hist = gamma.rvs(hist_gamma_shape, scale=hist_gamma_scale, size=n_samples)
    end = gamma.rvs(end_gamma_shape, scale=end_gamma_scale, size=n_samples)
    # Compare with KS test
    p = ks_2samp(hist, end).pvalue
    return p

In [10]:
### Compare each of the precipitation type distributions during times that have at least two types of frozen precipitation
### that span over 5000 grid points and test for significant changes at all points
# Use xr.apply_ufunc to perform the calculations over all spatial points
Snowfall_5000_p_values = xr.apply_ufunc(
    sample_and_test,
    Hist_5000_Snowfall_gamma_shape, Hist_5000_Snowfall_gamma_scale,
    End_5000_Snowfall_gamma_shape, End_5000_Snowfall_gamma_scale,
    kwargs={'n_samples': 1000},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float])
Rain_5000_p_values = xr.apply_ufunc(
    sample_and_test,
    Hist_5000_Rain_gamma_shape, Hist_5000_Rain_gamma_scale,
    End_5000_Rain_gamma_shape, End_5000_Rain_gamma_scale,
    kwargs={'n_samples': 1000},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float])
Ice_5000_p_values = xr.apply_ufunc(
    sample_and_test,
    Hist_5000_Ice_gamma_shape, Hist_5000_Ice_gamma_scale,
    End_5000_Ice_gamma_shape, End_5000_Ice_gamma_scale,
    kwargs={'n_samples': 1000},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float])
Fzra_5000_p_values = xr.apply_ufunc(
    sample_and_test,
    Hist_5000_Fzra_gamma_shape, Hist_5000_Fzra_gamma_scale,
    End_5000_Fzra_gamma_shape, End_5000_Fzra_gamma_scale,
    kwargs={'n_samples': 1000},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float])

Snowfall_5000_pdfchange_significant = Snowfall_5000_p_values < 0.05
Rain_5000_pdfchange_significant = Rain_5000_p_values < 0.05
Ice_5000_pdfchange_significant = Ice_5000_p_values < 0.05
Fzra_5000_pdfchange_significant = Fzra_5000_p_values < 0.05

In [12]:
### Compute distribution statistics for each precipitation type during times that have at least two types of frozen precipitation 
### that span over 5000 grid points and each period to plot changes
from scipy.stats import gamma

# Calculate the 90th percentile precipitation type accumulation and put in it xarray format
Snowfall_5000_Hist_q90 = xr.DataArray(gamma.ppf(0.9, Hist_5000_Snowfall_gamma_shape, scale=Hist_5000_Snowfall_gamma_scale),coords=Hist_5000_Snowfall_gamma_shape.coords,dims=Hist_5000_Snowfall_gamma_shape.dims)
Snowfall_5000_End_q90 = xr.DataArray(gamma.ppf(0.9, End_5000_Snowfall_gamma_shape, scale=End_5000_Snowfall_gamma_scale),coords=End_5000_Snowfall_gamma_shape.coords,dims=End_5000_Snowfall_gamma_shape.dims)
Rain_5000_Hist_q90 = xr.DataArray(gamma.ppf(0.9, Hist_5000_Rain_gamma_shape, scale=Hist_5000_Rain_gamma_scale),coords=Hist_5000_Rain_gamma_shape.coords,dims=Hist_5000_Rain_gamma_shape.dims)
Rain_5000_End_q90 = xr.DataArray(gamma.ppf(0.9, End_5000_Rain_gamma_shape, scale=End_5000_Rain_gamma_scale),coords=End_5000_Rain_gamma_shape.coords,dims=End_5000_Rain_gamma_shape.dims)
Ice_5000_Hist_q90 = xr.DataArray(gamma.ppf(0.9, Hist_5000_Ice_gamma_shape, scale=Hist_5000_Ice_gamma_scale),coords=Hist_5000_Ice_gamma_shape.coords,dims=Hist_5000_Ice_gamma_shape.dims)
Ice_5000_End_q90 = xr.DataArray(gamma.ppf(0.9, End_5000_Ice_gamma_shape, scale=End_5000_Ice_gamma_scale),coords=End_5000_Ice_gamma_shape.coords,dims=End_5000_Ice_gamma_shape.dims)
Fzra_5000_Hist_q90 = xr.DataArray(gamma.ppf(0.9, Hist_5000_Fzra_gamma_shape, scale=Hist_5000_Fzra_gamma_scale),coords=Hist_5000_Fzra_gamma_shape.coords,dims=Hist_5000_Fzra_gamma_shape.dims)
Fzra_5000_End_q90 = xr.DataArray(gamma.ppf(0.9, End_5000_Fzra_gamma_shape, scale=End_5000_Fzra_gamma_scale),coords=End_5000_Fzra_gamma_shape.coords,dims=End_5000_Fzra_gamma_shape.dims)

In [14]:
### Create a 4 panel plot of the distribution 90th percentile changes in each precipitation type across the domain for dual
### frozen precipitation 5000 grid point times
# Calculate the change in the distribution 90th percentiles for each precipitation type
Snowfall_5000_q90_change = Snowfall_5000_End_q90 - Snowfall_5000_Hist_q90
Rain_5000_q90_change = Rain_5000_End_q90 - Rain_5000_Hist_q90
Ice_5000_q90_change = Ice_5000_End_q90 - Ice_5000_Hist_q90
Fzra_5000_q90_change = Fzra_5000_End_q90 - Fzra_5000_Hist_q90

fig = plt.figure(figsize=(10, 12))

projection = ccrs.PlateCarree()

# Define axes manually
ax1 = fig.add_axes([0.08, 0.66, 0.38, 0.35], projection=projection)  # Top-left
ax2 = fig.add_axes([0.55, 0.66, 0.38, 0.35], projection=projection)  # Top-right
ax3 = fig.add_axes([0.08, 0.45, 0.38, 0.35], projection=projection)  # Bottom-left
ax4 = fig.add_axes([0.55, 0.45, 0.38, 0.35], projection=projection)  # Bottom-right

######## Snowfall 90th percentile change
# Add features
ax1.add_feature(cfeature.COASTLINE)
ax1.add_feature(cfeature.BORDERS)
ax1.add_feature(cfeature.STATES)
gl = ax1.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the 90th percentile snowfall change
snowfallq90_change_spatial = ax1.pcolormesh(Snowfall_5000_q90_change['lon'], Snowfall_5000_q90_change['lat'], Snowfall_5000_q90_change, 
                                            transform=projection,cmap='coolwarm', vmin=-30, vmax=30)

fig.colorbar(snowfallq90_change_spatial, ax=ax1, label='Change in 6-Hourly\n90th Percentile (mm)', shrink=0.4)
ax1.set_title('Change in 90th Percentile Snowfall Intensity', fontsize=10)

######## Rain 90th percentile change
# Add features
ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.BORDERS)
ax2.add_feature(cfeature.STATES)
gl = ax2.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the 90th percentile rain change
rainq90_change_spatial = ax2.pcolormesh(Rain_5000_q90_change['lon'], Rain_5000_q90_change['lat'], Rain_5000_q90_change, transform=projection,
                                cmap='coolwarm', vmin=-10, vmax=10)

fig.colorbar(rainq90_change_spatial, ax=ax2, label='Change in 6-Hourly\n90th Percentile (mm)', shrink=0.4)
ax2.set_title('Change in 90th Percentile Rainfall Intensity', fontsize=10)

######## Ice 90th percentile change
# Add features
ax3.add_feature(cfeature.COASTLINE)
ax3.add_feature(cfeature.BORDERS)
ax3.add_feature(cfeature.STATES)
gl = ax3.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the 90th percentile ice change
iceq90_change_spatial = ax3.pcolormesh(Ice_5000_q90_change['lon'], Ice_5000_q90_change['lat'], Ice_5000_q90_change, transform=projection,
                                cmap='coolwarm', vmin=-4, vmax=4)

fig.colorbar(iceq90_change_spatial, ax=ax3, label='Change in 6-Hourly\n90th Percentile (mm)', shrink=0.4)
ax3.set_title('Change in 90th Percentile Ice Intensity', fontsize=10)

######## Freezing Rain 90th percentile change
# Add features
ax4.add_feature(cfeature.COASTLINE)
ax4.add_feature(cfeature.BORDERS)
ax4.add_feature(cfeature.STATES)
gl = ax4.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the 90th percentile freezing rain change
fzraq90_change_spatial = ax4.pcolormesh(Fzra_5000_q90_change['lon'], Fzra_5000_q90_change['lat'], Fzra_5000_q90_change, transform=projection,
                                cmap='coolwarm', vmin=-5, vmax=5)

fig.colorbar(fzraq90_change_spatial, ax=ax4, label='Change in 6-Hourly\n90th Percentile (mm)', shrink=0.4)
ax4.set_title('Change in 90th Percentile Freezing Rain Intensity', fontsize=10)

# Add panel labels
ax1.text(0.01, 0.98, '(a)', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax2.text(0.01, 0.98, '(b)', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax3.text(0.01, 0.98, '(c)', transform=ax3.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax4.text(0.01, 0.98, '(d)', transform=ax4.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

plt.suptitle('Changes in the 90th Percentile Precipitation Type Intensity During Times With\nAt Least Two Types of Frozen Precipitation Spanning 5000 Grid Points\nEnd of Century - Historical')

plt.savefig('Key Figures/Changes in 90th Percentile Precipitation Type Intensity During 5000 Grid Point Dual Precipitation Times',bbox_inches = 'tight')

plt.close()

---------------------------------------------------------------------------------------------------------------
Grouping the 6-hourly 5000 Grid Point Precipitation Type Accumulations Into Continuous Events Based on Consecutive Time and Space
---------------------------------------------------------------------------------------------------------------

In [2]:
### Read in the precipitation data
HistoricalPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_Hist_Precip_Accumulations.nc', engine = 'netcdf4')
EndPrecipData = xr.open_dataset('/glade/derecho/scratch/slaurina/Statistics Project/Northeast_End_Precip_Accumulations.nc', engine = 'netcdf4')
HistoricalPrecipData

<xarray.Dataset> Size: 87GB
Dimensions:         (Time: 21847, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 175kB 1990-10-01 ... 2005-09-30
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 17GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 17GB ...
    Rain_Accum      (Time, south_north, west_east) float32 17GB ...
    Ice_Accum       (Time, south_north, west_east) float32 17GB ...
    Fzra_Accum      (Time, south_north, west_east) float32 17GB ...

In [3]:
### Find times in the data where there is freezing precipitation of at least two types greater than a certain value
## Historical
# Create masks for freezing precipitation types above the threshold
H_fzra_mask = HistoricalPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_ice_mask = HistoricalPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
H_snow_mask = HistoricalPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
H_fzra_count = H_fzra_mask.sum(dim=["south_north", "west_east"])
H_ice_count = H_ice_mask.sum(dim=["south_north", "west_east"])
H_snow_count = H_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the space threshold at each time step
H_freezing_precip5000_count = ((H_fzra_count >= 5000).astype(int) + (H_ice_count >= 5000).astype(int) + (H_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their space thresholds
H_valid_times = HistoricalPrecipData["Time"].where(H_freezing_precip5000_count >= 2, drop=True)
print('Number of Historical Times With at Least Two Types of Frozen Precipitation:', len(H_valid_times['Time']))

## End of Century
# Create masks for freezing precipitation types above the threshold
E_fzra_mask = EndPrecipData["Fzra_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_ice_mask = EndPrecipData["Ice_Accum"] > 0.254 # Greater than 0.01 inches or 0.254 mm
E_snow_mask = EndPrecipData["Snowfall_Accum"] > 2.54 # Greater than 0.1 inches or 2.54 mm

# Count grid points where each type exceeds its threshold at each time step
E_fzra_count = E_fzra_mask.sum(dim=["south_north", "west_east"])
E_ice_count = E_ice_mask.sum(dim=["south_north", "west_east"])
E_snow_count = E_snow_mask.sum(dim=["south_north", "west_east"])

# Sum the number of freezing precip types exceeding the threshold at each time step
E_freezing_precip5000_count = ((E_fzra_count >= 5000).astype(int) + (E_ice_count >= 5000).astype(int) + (E_snow_count >= 5000).astype(int))

# Find times where at least 2 types of freezing precip exceed their thresholds
E_valid_times = EndPrecipData["Time"].where(E_freezing_precip5000_count >= 2, drop=True)
print('Number of End of Century Times With at Least Two Types of Frozen Precipitation:', len(E_valid_times['Time']))

Number of Historical Times With at Least Two Types of Frozen Precipitation: 3111
Number of End of Century Times With at Least Two Types of Frozen Precipitation: 2110


In [4]:
### Subset the data for only the times with at least two types of frozen precipitation that span over 5000 grid points each
H_precip_type_5000_data = HistoricalPrecipData.sel(Time=H_valid_times)
E_precip_type_5000_data = EndPrecipData.sel(Time=E_valid_times)
H_precip_type_5000_data

<xarray.Dataset> Size: 12GB
Dimensions:         (Time: 3111, south_north: 360, west_east: 550)
Coordinates:
  * Time            (Time) datetime64[ns] 25kB 1990-11-05T12:00:00 ... 2005-0...
    lat             (south_north, west_east) float32 792kB ...
    lon             (south_north, west_east) float32 792kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    Snow_Accum      (Time, south_north, west_east) float32 2GB ...
    Snowfall_Accum  (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Rain_Accum      (Time, south_north, west_east) float32 2GB ...
    Ice_Accum       (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0
    Fzra_Accum      (Time, south_north, west_east) float32 2GB 0.0 0.0 ... 0.0

In [5]:
### Subset into each precipitation type for each period's times that have at least two types of frozen precipitation that span over 5000 grid points each
# Historical variables
Hist_5000_6hourly_Snowfall = H_precip_type_5000_data['Snowfall_Accum']
Hist_5000_6hourly_Rain = H_precip_type_5000_data['Rain_Accum']
Hist_5000_6hourly_Ice = H_precip_type_5000_data['Ice_Accum']
Hist_5000_6hourly_Fzra = H_precip_type_5000_data['Fzra_Accum']

# End of century variables
End_5000_6hourly_Snowfall = E_precip_type_5000_data['Snowfall_Accum']
End_5000_6hourly_Rain = E_precip_type_5000_data['Rain_Accum']
End_5000_6hourly_Ice = E_precip_type_5000_data['Ice_Accum']
End_5000_6hourly_Fzra = E_precip_type_5000_data['Fzra_Accum']

In [6]:
### Grouping the 6-hourly historical snowfalls into continuous events based on how many grid points have snowfall for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous snow blobs
for t in Hist_5000_6hourly_Snowfall.Time.values:
    # Get snowfall at that time step
    snow_t = Hist_5000_6hourly_Snowfall.sel(Time=t)
    
    # Apply threshold to get snow mask
    snow_mask = snow_t > 2.54 # greater than 0.1 inches (2.54mm)
    
    # Apply connected component labeling
    labeled, num_features = label(snow_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events (same as your logic)
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata (same as your code)
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_snowfall_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_snow = Hist_5000_6hourly_Snowfall.sel(Time=times)
    total_snow = event_snow.sum(dim='Time')
    event_snowfall_list.append(total_snow)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_snowfall_all = xr.concat(event_snowfall_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

H_snowfall_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_snowfall_per_event": event_snowfall_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

H_snowfall_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 665MB
Dimensions:                   (Time: 3017, south_north: 360, west_east: 550,
                               Event: 838)
Coordinates:
  * Time                      (Time) datetime64[ns] 24kB 1990-11-05T12:00:00 ...
    lat                       (south_north, west_east) float32 792kB 38.28 .....
    lon                       (south_north, west_east) float32 792kB -93.16 ....
  * Event                     (Event) int64 7kB 1 2 3 4 5 ... 835 836 837 838
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id                  (Time) int64 24kB 1 2 2 3 3 ... 837 838 838 838
    Total_snowfall_per_event  (Event, south_north, west_east) float32 664MB 0...
    Event_duration            (Event) int64 7kB 6 12 24 6 24 ... 12 54 12 18 18
    Event_start               (Event) datetime64[ns] 7kB 1990-11-05T12:00:00 ...
    Event_end                 (Event) datetime64[ns] 7kB 1990-11-05T12:00:00 ...

In [7]:
### Grouping the 6-hourly end of century snowfalls into continuous events based on how many grid points have snowfall for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous snow blobs
for t in End_5000_6hourly_Snowfall.Time.values:
    # Get snowfall at that time step
    snow_t = End_5000_6hourly_Snowfall.sel(Time=t)
    
    # Apply threshold to get snow mask
    snow_mask = snow_t > 2.54 # greater than 0.1 inches (2.54mm)
    
    # Apply connected component labeling
    labeled, num_features = label(snow_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events (same as your logic)
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata (same as your code)
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_snowfall_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_snow = End_5000_6hourly_Snowfall.sel(Time=times)
    total_snow = event_snow.sum(dim='Time')
    event_snowfall_list.append(total_snow)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_snowfall_all = xr.concat(event_snowfall_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

E_snowfall_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_snowfall_per_event": event_snowfall_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

E_snowfall_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 444MB
Dimensions:                   (Time: 2023, south_north: 360, west_east: 550,
                               Event: 558)
Coordinates:
  * Time                      (Time) datetime64[ns] 16kB 2085-11-29T12:00:00 ...
    lat                       (south_north, west_east) float32 792kB 38.28 .....
    lon                       (south_north, west_east) float32 792kB -93.16 ....
  * Event                     (Event) int64 4kB 1 2 3 4 5 ... 555 556 557 558
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id                  (Time) int64 16kB 1 1 1 1 2 ... 556 557 558 558
    Total_snowfall_per_event  (Event, south_north, west_east) float32 442MB 0...
    Event_duration            (Event) int64 4kB 24 12 66 6 60 24 ... 6 6 12 6 12
    Event_start               (Event) datetime64[ns] 4kB 2085-11-29T12:00:00 ...
    Event_end                 (Event) datetime64[ns] 4kB 2085-11-30T06:00:00 ...

In [8]:
### Grouping the 6-hourly historical rainfalls into continuous events based on how many grid points have rainfall for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous rain blobs
for t in Hist_5000_6hourly_Rain.Time.values:
    # Get rainfall at that time step
    rain_t = Hist_5000_6hourly_Rain.sel(Time=t)
    
    # Apply threshold to get rain mask
    rain_mask = rain_t > 2.54 # greater than 0.1 inches (2.54mm)
    
    # Apply connected component labeling
    labeled, num_features = label(rain_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_rainfall_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_rain = Hist_5000_6hourly_Rain.sel(Time=times)
    total_rain = event_rain.sum(dim='Time')
    event_rainfall_list.append(total_rain)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_rainfall_all = xr.concat(event_rainfall_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

H_rainfall_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_rainfall_per_event": event_rainfall_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

H_rainfall_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 406MB
Dimensions:                   (Time: 1525, south_north: 360, west_east: 550,
                               Event: 510)
Coordinates:
  * Time                      (Time) datetime64[ns] 12kB 1990-11-06T18:00:00 ...
    lat                       (south_north, west_east) float32 792kB 38.28 .....
    lon                       (south_north, west_east) float32 792kB -93.16 ....
  * Event                     (Event) int64 4kB 1 2 3 4 5 ... 507 508 509 510
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id                  (Time) int64 12kB 1 2 2 2 2 ... 509 510 510 510
    Total_rainfall_per_event  (Event, south_north, west_east) float32 404MB 0...
    Event_duration            (Event) int64 4kB 6 30 6 12 42 ... 6 12 12 18 18
    Event_start               (Event) datetime64[ns] 4kB 1990-11-06T18:00:00 ...
    Event_end                 (Event) datetime64[ns] 4kB 1990-11-06T18:00:00 ...

In [9]:
### Grouping the 6-hourly end of century rainfalls into continuous events based on how many grid points have rainfall for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous rain blobs
for t in End_5000_6hourly_Rain.Time.values:
    # Get rainfall at that time step
    rain_t = End_5000_6hourly_Rain.sel(Time=t)
    
    # Apply threshold to get rain mask
    rain_mask = rain_t > 2.54 # greater than 0.1 inches (2.54mm)
    
    # Apply connected component labeling
    labeled, num_features = label(rain_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_rainfall_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_rain = End_5000_6hourly_Rain.sel(Time=times)
    total_rain = event_rain.sum(dim='Time')
    event_rainfall_list.append(total_rain)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_rainfall_all = xr.concat(event_rainfall_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

E_rainfall_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_rainfall_per_event": event_rainfall_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

E_rainfall_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 291MB
Dimensions:                   (Time: 1049, south_north: 360, west_east: 550,
                               Event: 365)
Coordinates:
  * Time                      (Time) datetime64[ns] 8kB 2085-11-29T12:00:00 ....
    lat                       (south_north, west_east) float32 792kB 38.28 .....
    lon                       (south_north, west_east) float32 792kB -93.16 ....
  * Event                     (Event) int64 3kB 1 2 3 4 5 ... 362 363 364 365
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id                  (Time) int64 8kB 1 1 1 1 2 ... 362 363 363 364 365
    Total_rainfall_per_event  (Event, south_north, west_east) float32 289MB 0...
    Event_duration            (Event) int64 3kB 24 12 48 24 12 ... 24 6 12 6 6
    Event_start               (Event) datetime64[ns] 3kB 2085-11-29T12:00:00 ...
    Event_end                 (Event) datetime64[ns] 3kB 2085-11-30T06:00:00 ...

In [10]:
### Grouping the 6-hourly historical ice accumulations into continuous events based on how many grid points have ice for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous ice blobs
for t in Hist_5000_6hourly_Ice.Time.values:
    # Get ice at that time step
    ice_t = Hist_5000_6hourly_Ice.sel(Time=t)
    
    # Apply threshold to get ice mask
    ice_mask = ice_t > 0.254 # greater than 0.01 inches (0.254mm)
    
    # Apply connected component labeling
    labeled, num_features = label(ice_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_ice_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_ice = Hist_5000_6hourly_Ice.sel(Time=times)
    total_ice = event_ice.sum(dim='Time')
    event_ice_list.append(total_ice)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_ice_all = xr.concat(event_ice_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

H_ice_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_ice_per_event": event_ice_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

H_ice_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 191MB
Dimensions:              (Time: 601, south_north: 360, west_east: 550,
                          Event: 239)
Coordinates:
  * Time                 (Time) datetime64[ns] 5kB 1990-11-21T12:00:00 ... 20...
    lat                  (south_north, west_east) float32 792kB 38.28 ... 46.57
    lon                  (south_north, west_east) float32 792kB -93.16 ... -6...
  * Event                (Event) int64 2kB 1 2 3 4 5 6 ... 235 236 237 238 239
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id             (Time) int64 5kB 1 1 2 2 2 2 ... 237 238 238 238 239
    Total_ice_per_event  (Event, south_north, west_east) float32 189MB 0.0 .....
    Event_duration       (Event) int64 2kB 12 42 6 6 6 6 12 ... 6 6 18 12 18 6
    Event_start          (Event) datetime64[ns] 2kB 1990-11-21T12:00:00 ... 2...
    Event_end            (Event) datetime64[ns] 2kB 1990-11-21T18:00:00 ... 2...

In [11]:
### Grouping the 6-hourly end of century ice accumulations into continuous events based on how many grid points have ice for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous ice blobs
for t in End_5000_6hourly_Ice.Time.values:
    # Get ice at that time step
    ice_t = End_5000_6hourly_Ice.sel(Time=t)
    
    # Apply threshold to get ice mask
    ice_mask = ice_t > 0.254 # greater than 0.01 inches (0.254mm)
    
    # Apply connected component labeling
    labeled, num_features = label(ice_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_ice_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_ice = End_5000_6hourly_Ice.sel(Time=times)
    total_ice = event_ice.sum(dim='Time')
    event_ice_list.append(total_ice)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_ice_all = xr.concat(event_ice_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

E_ice_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_ice_per_event": event_ice_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

E_ice_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 120MB
Dimensions:              (Time: 331, south_north: 360, west_east: 550,
                          Event: 149)
Coordinates:
  * Time                 (Time) datetime64[ns] 3kB 2085-12-13T06:00:00 ... 21...
    lat                  (south_north, west_east) float32 792kB 38.28 ... 46.57
    lon                  (south_north, west_east) float32 792kB -93.16 ... -6...
  * Event                (Event) int64 1kB 1 2 3 4 5 6 ... 145 146 147 148 149
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id             (Time) int64 3kB 1 1 1 1 1 1 ... 148 148 148 149 149
    Total_ice_per_event  (Event, south_north, west_east) float32 118MB 0.0 .....
    Event_duration       (Event) int64 1kB 36 18 6 12 6 24 ... 6 30 12 6 24 12
    Event_start          (Event) datetime64[ns] 1kB 2085-12-13T06:00:00 ... 2...
    Event_end            (Event) datetime64[ns] 1kB 2085-12-14T12:00:00 ... 2...

In [12]:
### Grouping the 6-hourly historical freezing rain accumulations into continuous events based on how many grid points have freezing rain for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous freezing rain blobs
for t in Hist_5000_6hourly_Fzra.Time.values:
    # Get freezing rain at that time step
    fzra_t = Hist_5000_6hourly_Fzra.sel(Time=t)
    
    # Apply threshold to get freezing rain mask
    fzra_mask = fzra_t > 0.254 # greater than 0.01 inches (0.254mm)
    
    # Apply connected component labeling
    labeled, num_features = label(fzra_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_fzra_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_fzra = Hist_5000_6hourly_Fzra.sel(Time=times)
    total_fzra = event_fzra.sum(dim='Time')
    event_fzra_list.append(total_fzra)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_fzra_all = xr.concat(event_fzra_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

H_fzra_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_fzra_per_event": event_fzra_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

H_fzra_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 512MB
Dimensions:               (Time: 1599, south_north: 360, west_east: 550,
                           Event: 644)
Coordinates:
  * Time                  (Time) datetime64[ns] 13kB 1990-11-06T12:00:00 ... ...
    lat                   (south_north, west_east) float32 792kB 38.28 ... 46.57
    lon                   (south_north, west_east) float32 792kB -93.16 ... -...
  * Event                 (Event) int64 5kB 1 2 3 4 5 6 ... 640 641 642 643 644
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id              (Time) int64 13kB 1 1 2 2 3 4 ... 641 642 643 644 644
    Total_fzra_per_event  (Event, south_north, west_east) float32 510MB 0.0 ....
    Event_duration        (Event) int64 5kB 12 12 6 12 12 42 6 ... 18 6 6 6 6 12
    Event_start           (Event) datetime64[ns] 5kB 1990-11-06T12:00:00 ... ...
    Event_end             (Event) datetime64[ns] 5kB 1990-11-06T18:00:00 ... ...

In [13]:
### Grouping the 6-hourly end of century freezing rain accumulations into continuous events based on how many grid points have freezing rain for dual precipitation 
### 5000 grid point times, but using connected 5000 grid point blobs as the criteria to see if there are any differences
from scipy.ndimage import label

# Blob size threshold
min_blob_size = 5000

# Store valid times (when a large enough contiguous blob exists)
valid_times = []

# Loop through each time step to check for large spatially-contiguous freezing rain blobs
for t in End_5000_6hourly_Fzra.Time.values:
    # Get freezing rain at that time step
    fzra_t = End_5000_6hourly_Fzra.sel(Time=t)
    
    # Apply threshold to get freezing rain mask
    fzra_mask = fzra_t > 0.254 # greater than 0.01 inches (0.254mm)
    
    # Apply connected component labeling
    labeled, num_features = label(fzra_mask)
    
    # Count grid points in each labeled blob
    blob_sizes = np.bincount(labeled.ravel())
    
    # Ignore label 0 (background)
    if (blob_sizes[1:] >= min_blob_size).any():
        valid_times.append(t)

# Now group valid times into continuous events
event_list = []
current_event = [valid_times[0]]

for i in range(1, len(valid_times)):
    if (valid_times[i] - valid_times[i - 1]) == np.timedelta64(6, 'h'):
        current_event.append(valid_times[i])
    else:
        event_list.append(current_event)
        current_event = [valid_times[i]]
if current_event:
    event_list.append(current_event)

# Assign event IDs and extract metadata
event_ids = np.arange(1, len(event_list) + 1)

event_time_list = []
event_id_list = []
event_fzra_list = []
event_duration_list = []
event_start_list = []
event_end_list = []

for event_id, times in zip(event_ids, event_list):
    event_time_list.extend(times)
    event_id_list.extend([event_id] * len(times))
    
    event_fzra = End_5000_6hourly_Fzra.sel(Time=times)
    total_fzra = event_fzra.sum(dim='Time')
    event_fzra_list.append(total_fzra)
    
    duration = len(times) * 6
    event_duration_list.append(duration)
    
    event_start_list.append(times[0])
    event_end_list.append(times[-1])

# Construct final xarray Dataset
event_classification = xr.DataArray(event_id_list, coords={"Time": event_time_list}, dims="Time", name="Event_id")
event_fzra_all = xr.concat(event_fzra_list, dim=xr.DataArray(event_ids, dims="Event", name="Event"))
event_duration_all = xr.DataArray(event_duration_list, coords={"Event": event_ids}, dims="Event", name="Event_duration")
event_start_all = xr.DataArray(event_start_list, coords={"Event": event_ids}, dims="Event", name="Event_start")
event_end_all = xr.DataArray(event_end_list, coords={"Event": event_ids}, dims="Event", name="Event_end")

E_fzra_5000_spatially_contiguous_event_dataset = xr.Dataset({
    "Event_id": event_classification,
    "Total_fzra_per_event": event_fzra_all,
    "Event_duration": event_duration_all,
    "Event_start": event_start_all,
    "Event_end": event_end_all})

E_fzra_5000_spatially_contiguous_event_dataset

<xarray.Dataset> Size: 353MB
Dimensions:               (Time: 974, south_north: 360, west_east: 550,
                           Event: 444)
Coordinates:
  * Time                  (Time) datetime64[ns] 8kB 2085-12-11T18:00:00 ... 2...
    lat                   (south_north, west_east) float32 792kB 38.28 ... 46.57
    lon                   (south_north, west_east) float32 792kB -93.16 ... -...
  * Event                 (Event) int64 4kB 1 2 3 4 5 6 ... 440 441 442 443 444
Dimensions without coordinates: south_north, west_east
Data variables:
    Event_id              (Time) int64 8kB 1 2 2 2 2 2 ... 442 443 443 444 444
    Total_fzra_per_event  (Event, south_north, west_east) float32 352MB 0.0 ....
    Event_duration        (Event) int64 4kB 6 42 12 18 6 30 ... 6 12 12 12 12 12
    Event_start           (Event) datetime64[ns] 4kB 2085-12-11T18:00:00 ... ...
    Event_end             (Event) datetime64[ns] 4kB 2085-12-11T18:00:00 ... ...

-------------------------------------------------------------------------------------------------------------------------------------------------------
Changes in the Average Classified Event Total Precipitation Accumulation During Classified Events With Two or More Frozen Precipitation Types Covering at Least 5000 Grid Points
-------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
### Calculate the average precipitation type accumulations spatially per classified event with dual precipitation 5000 grid point times
# Historical
H_snowfall_event_total_average = H_snowfall_5000_spatially_contiguous_event_dataset['Total_snowfall_per_event'].mean(dim='Event')
H_rainfall_event_total_average = H_rainfall_5000_spatially_contiguous_event_dataset['Total_rainfall_per_event'].mean(dim='Event')
H_ice_event_total_average = H_ice_5000_spatially_contiguous_event_dataset['Total_ice_per_event'].mean(dim='Event')
H_fzra_event_total_average = H_fzra_5000_spatially_contiguous_event_dataset['Total_fzra_per_event'].mean(dim='Event')

# End of century
E_snowfall_event_total_average = E_snowfall_5000_spatially_contiguous_event_dataset['Total_snowfall_per_event'].mean(dim='Event')
E_rainfall_event_total_average = E_rainfall_5000_spatially_contiguous_event_dataset['Total_rainfall_per_event'].mean(dim='Event')
E_ice_event_total_average = E_ice_5000_spatially_contiguous_event_dataset['Total_ice_per_event'].mean(dim='Event')
E_fzra_event_total_average = E_fzra_5000_spatially_contiguous_event_dataset['Total_fzra_per_event'].mean(dim='Event')

# Calculate the change in average precipitation type accumulation per event
event_snowfall_total_average_change = E_snowfall_event_total_average - H_snowfall_event_total_average
event_rainfall_total_average_change = E_rainfall_event_total_average - H_rainfall_event_total_average
event_ice_total_average_change = E_ice_event_total_average - H_ice_event_total_average
event_fzra_total_average_change = E_fzra_event_total_average - H_fzra_event_total_average

In [15]:
### Create a 4 panel plot of the changes in the average classified event total preciptiation for each precipitation type during
### dual frozen precipitation 5000 grid point times
fig = plt.figure(figsize=(10, 12))

projection = ccrs.PlateCarree()

# Define axes manually
ax1 = fig.add_axes([0.08, 0.66, 0.38, 0.35], projection=projection)  # Top-left (Snowfall event average changes)
ax2 = fig.add_axes([0.55, 0.66, 0.38, 0.35], projection=projection)  # Top-right (Rain event average changes)
ax3 = fig.add_axes([0.08, 0.45, 0.38, 0.35], projection=projection)  # Bottom-left (Ice event average changes)
ax4 = fig.add_axes([0.55, 0.45, 0.38, 0.35], projection=projection)  # Bottom-right (Freezing rain event average changes)

######## Snowfall average event total change
# Add features
ax1.add_feature(cfeature.COASTLINE)
ax1.add_feature(cfeature.BORDERS)
ax1.add_feature(cfeature.STATES)
gl = ax1.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total snowfall change
snowfall_total_average_change_spatial = ax1.pcolormesh(event_snowfall_total_average_change['lon'], event_snowfall_total_average_change['lat'], 
                                            event_snowfall_total_average_change, transform=projection, cmap='coolwarm', vmin=-30, vmax=30)

fig.colorbar(snowfall_total_average_change_spatial, ax=ax1, label='Change in Average (mm)', shrink=0.4)
ax1.set_title('Change in Average Event Total Snowfall', fontsize=10)

######## Rain average event total change
# Add features
ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.BORDERS)
ax2.add_feature(cfeature.STATES)
gl = ax2.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total rain change
rain_total_average_change_spatial = ax2.pcolormesh(event_rainfall_total_average_change['lon'], event_rainfall_total_average_change['lat'], 
                                            event_rainfall_total_average_change, transform=projection, cmap='coolwarm', vmin=-5, vmax=5)

fig.colorbar(rain_total_average_change_spatial, ax=ax2, label='Change in Average (mm)', shrink=0.4)
ax2.set_title('Change in Average Event Total Rainfall', fontsize=10)

######## Ice average event total change
# Add features
ax3.add_feature(cfeature.COASTLINE)
ax3.add_feature(cfeature.BORDERS)
ax3.add_feature(cfeature.STATES)
gl = ax3.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total ice change
ice_total_average_change_spatial = ax3.pcolormesh(event_ice_total_average_change['lon'], event_ice_total_average_change['lat'], 
                                            event_ice_total_average_change, transform=projection, cmap='coolwarm', vmin=-1, vmax=1)

fig.colorbar(ice_total_average_change_spatial, ax=ax3, label='Change in Average (mm)', shrink=0.4)
ax3.set_title('Change in Average Event Total Ice', fontsize=10)

######## Freezing Rain average event total change
# Add features
ax4.add_feature(cfeature.COASTLINE)
ax4.add_feature(cfeature.BORDERS)
ax4.add_feature(cfeature.STATES)
gl = ax4.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total freezing rain change
fzra_total_average_change_spatial = ax4.pcolormesh(event_fzra_total_average_change['lon'], event_fzra_total_average_change['lat'], 
                                            event_fzra_total_average_change, transform=projection, cmap='coolwarm', vmin=-1, vmax=1)

fig.colorbar(fzra_total_average_change_spatial, ax=ax4, label='Change in Average (mm)', shrink=0.4)
ax4.set_title('Change in Average Event Total Freezing Rain', fontsize=10)

# Add panel labels
ax1.text(0.01, 0.98, '(a)', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax2.text(0.01, 0.98, '(b)', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax3.text(0.01, 0.98, '(c)', transform=ax3.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax4.text(0.01, 0.98, '(d)', transform=ax4.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

plt.suptitle('Changes in Average Classified Event Total Precipitation Type During Times With\nAt Least Two Types of Frozen Precipitation Spanning 5000 Grid Points\nEnd of Century - Historical')

plt.savefig('Key Figures/Changes in Average Event Total Precipitation During 5000 Grid Point Dual Precipitation Times',bbox_inches = 'tight')

plt.close()

---------------------------------------------------------------------------------------------------------------------------------------------------
Changes in the Event Duration Distributions During Classified Events with Two or More Frozen Precipitation Types Covering at Least 5000 Grid Points
---------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
### Get the duration data ready to be plotted as histograms
# Extract all the event durations pandas series
hist_snowfall_event_durations = H_snowfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_snowfall_event_durations = E_snowfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
hist_rainfall_event_durations = H_rainfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_rainfall_event_durations = E_rainfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
hist_ice_event_durations = H_ice_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_ice_event_durations = E_ice_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
hist_fzra_event_durations = H_fzra_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_fzra_event_durations = E_fzra_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()

# Total number of classified snowfall events
unique_snowfall_h_events = len(H_snowfall_5000_spatially_contiguous_event_dataset['Event'])
unique_snowfall_e_events = len(E_snowfall_5000_spatially_contiguous_event_dataset['Event'])
unique_rainfall_h_events = len(H_rainfall_5000_spatially_contiguous_event_dataset['Event'])
unique_rainfall_e_events = len(E_rainfall_5000_spatially_contiguous_event_dataset['Event'])
unique_ice_h_events = len(H_ice_5000_spatially_contiguous_event_dataset['Event'])
unique_ice_e_events = len(E_ice_5000_spatially_contiguous_event_dataset['Event'])
unique_fzra_h_events = len(H_fzra_5000_spatially_contiguous_event_dataset['Event'])
unique_fzra_e_events = len(E_fzra_5000_spatially_contiguous_event_dataset['Event'])

# Get unique event durations and their counts
hist_snowfall_counts = hist_snowfall_event_durations.value_counts().sort_index()
end_snowfall_counts = end_snowfall_event_durations.value_counts().sort_index()
hist_rainfall_counts = hist_rainfall_event_durations.value_counts().sort_index()
end_rainfall_counts = end_rainfall_event_durations.value_counts().sort_index()
hist_ice_counts = hist_ice_event_durations.value_counts().sort_index()
end_ice_counts = end_ice_event_durations.value_counts().sort_index()
hist_fzra_counts = hist_fzra_event_durations.value_counts().sort_index()
end_fzra_counts = end_fzra_event_durations.value_counts().sort_index()

# Reindex to fill missing durations with zero counts
hist_snowfall_counts = hist_snowfall_counts.reindex(hist_snowfall_event_durations, fill_value=0)
end_snowfall_counts = end_snowfall_counts.reindex(end_snowfall_event_durations, fill_value=0)
hist_rainfall_counts = hist_rainfall_counts.reindex(hist_rainfall_event_durations, fill_value=0)
end_rainfall_counts = end_rainfall_counts.reindex(end_rainfall_event_durations, fill_value=0)
hist_ice_counts = hist_ice_counts.reindex(hist_ice_event_durations, fill_value=0)
end_ice_counts = end_ice_counts.reindex(end_ice_event_durations, fill_value=0)
hist_fzra_counts = hist_fzra_counts.reindex(hist_fzra_event_durations, fill_value=0)
end_fzra_counts = end_fzra_counts.reindex(end_fzra_event_durations, fill_value=0)

# Calculate the 90th percentile durations for each period
snowfall_perc90_hist = np.percentile(hist_snowfall_event_durations, 90)
snowfall_perc90_end = np.percentile(end_snowfall_event_durations, 90)
rainfall_perc90_hist = np.percentile(hist_rainfall_event_durations, 90)
rainfall_perc90_end = np.percentile(end_rainfall_event_durations, 90)
ice_perc90_hist = np.percentile(hist_ice_event_durations, 90)
ice_perc90_end = np.percentile(end_ice_event_durations, 90)
fzra_perc90_hist = np.percentile(hist_fzra_event_durations, 90)
fzra_perc90_end = np.percentile(end_fzra_event_durations, 90)

In [20]:
### Print out the 90th percentile distributions
print(snowfall_perc90_hist)
print(snowfall_perc90_end)
print(rainfall_perc90_hist)
print(rainfall_perc90_end)
print(ice_perc90_hist)
print(ice_perc90_end)
print(fzra_perc90_hist)
print(fzra_perc90_end)

48.0
48.0
42.0
36.0
30.0
24.0
30.0
30.0


In [26]:
### Create a 4 panel plot of the changes in the classified event duration distributions for each precipitation type and each period during
### dual frozen precipitation 5000 grid point times
# Define the ticks for the plot
tick_durations = np.arange(0,241,24)

# Define bar width
bar_width = 6  # Keeps bars from overlapping too much

fig = plt.figure(figsize=(10, 12))

# Define axes manually
ax1 = fig.add_axes([0.08, 0.69, 0.42, 0.20])  # Top-left (Snowfall event duration changes)
ax2 = fig.add_axes([0.58, 0.69, 0.42, 0.20])  # Top-right (Rain event duration changes)
ax3 = fig.add_axes([0.08, 0.41, 0.42, 0.20])  # Bottom-left (Ice event duration changes)
ax4 = fig.add_axes([0.58, 0.41, 0.42, 0.20])  # Bottom-right (Freezing rain event duration changes)

######## Snowfall event duration distribution changes
# Plot the duration distributions
ax1.bar(hist_snowfall_counts.index, hist_snowfall_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_snowfall_h_events})')
ax1.bar(end_snowfall_counts.index, end_snowfall_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_snowfall_e_events})')

# Add vertical lines for the 90th percentile duration
ax1.axvline(snowfall_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax1.axvline(snowfall_perc90_end, color='red', linestyle='--', linewidth=2)

ax1.set_xticks(tick_durations)
ax1.set_xticklabels(tick_durations)
ax1.set_xlim(0,240)
ax1.set_xlabel('Hours')
ax1.set_ylabel('# of Events')
ax1.set_yscale('log')
ax1.set_ylim(0.9,300)
ax1.set_title('Distribution of Snowfall Event Durations', fontsize=12)
ax1.legend(fontsize=10)

######## Rainfall event duration distribution changes
# Plot the duration distributions
ax2.bar(hist_rainfall_counts.index, hist_rainfall_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_rainfall_h_events})')
ax2.bar(end_rainfall_counts.index, end_rainfall_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_rainfall_e_events})')

# Add vertical lines for the 90th percentile duration
ax2.axvline(rainfall_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax2.axvline(rainfall_perc90_end, color='red', linestyle='--', linewidth=2)

ax2.set_xticks(tick_durations)
ax2.set_xticklabels(tick_durations)
ax2.set_xlim(0,240)
ax2.set_xlabel('Hours')
ax2.set_ylabel('# of Events')
ax2.set_yscale('log')
ax2.set_ylim(0.9,300)
ax2.set_title('Distribution of Rainfall Event Durations', fontsize=12)
ax2.legend(fontsize=10)

######## Ice event duration distribution changes
# Plot the duration distributions
ax3.bar(hist_ice_counts.index, hist_ice_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_ice_h_events})')
ax3.bar(end_ice_counts.index, end_ice_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_ice_e_events})')

# Add vertical lines for the 90th percentile duration
ax3.axvline(ice_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax3.axvline(ice_perc90_end, color='red', linestyle='--', linewidth=2)

ax3.set_xticks(tick_durations)
ax3.set_xticklabels(tick_durations)
ax3.set_xlim(0,240)
ax3.set_xlabel('Hours')
ax3.set_ylabel('# of Events')
ax3.set_yscale('log')
ax3.set_ylim(0.9,300)
ax3.set_title('Distribution of Ice Event Durations', fontsize=12)
ax3.legend(fontsize=10)

######## Freezing rain event duration distribution changes
# Plot the duration distributions
ax4.bar(hist_fzra_counts.index, hist_fzra_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_fzra_h_events})')
ax4.bar(end_fzra_counts.index, end_fzra_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_fzra_e_events})')

# Add vertical lines for the 90th percentile duration
ax4.axvline(fzra_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax4.axvline(fzra_perc90_end, color='red', linestyle='--', linewidth=2)

ax4.set_xticks(tick_durations)
ax4.set_xticklabels(tick_durations)
ax4.set_xlim(0,240)
ax4.set_xlabel('Hours')
ax4.set_ylabel('# of Events')
ax4.set_yscale('log')
ax4.set_ylim(0.9,300)
ax4.set_title('Distribution of Freezing Rain Event Durations', fontsize=12)
ax4.legend(fontsize=10)

# Add panel labels
ax1.text(0.04, 0.98, '(a)', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax2.text(0.04, 0.98, '(b)', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax3.text(0.04, 0.98, '(c)', transform=ax3.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax4.text(0.04, 0.98, '(d)', transform=ax4.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

plt.suptitle('Comparison of Classified Precipitation Type Event Duration Distributions During Times With\nAt Least Two Types of Frozen Precipitation Spanning 5000 Grid Points')

plt.savefig('Key Figures/Changes in Classified Event Duration Distributions During 5000 Grid Point Dual Precipitation Times',bbox_inches = 'tight')

plt.close()

---------------------------------------------------------------------------------------------------------------------------------------------------
Changes in the Average Classified Event Total Precipitation Accumulation and Event Duration Distributions During Classified Events with Two or More Frozen Precipitation Types Covering at Least 5000 Grid Points
---------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
### Calculate the average precipitation type accumulations spatially per classified event with dual precipitation 5000 grid point times
# Historical
H_snowfall_event_total_average = H_snowfall_5000_spatially_contiguous_event_dataset['Total_snowfall_per_event'].mean(dim='Event')
H_rainfall_event_total_average = H_rainfall_5000_spatially_contiguous_event_dataset['Total_rainfall_per_event'].mean(dim='Event')
H_ice_event_total_average = H_ice_5000_spatially_contiguous_event_dataset['Total_ice_per_event'].mean(dim='Event')
H_fzra_event_total_average = H_fzra_5000_spatially_contiguous_event_dataset['Total_fzra_per_event'].mean(dim='Event')

# End of century
E_snowfall_event_total_average = E_snowfall_5000_spatially_contiguous_event_dataset['Total_snowfall_per_event'].mean(dim='Event')
E_rainfall_event_total_average = E_rainfall_5000_spatially_contiguous_event_dataset['Total_rainfall_per_event'].mean(dim='Event')
E_ice_event_total_average = E_ice_5000_spatially_contiguous_event_dataset['Total_ice_per_event'].mean(dim='Event')
E_fzra_event_total_average = E_fzra_5000_spatially_contiguous_event_dataset['Total_fzra_per_event'].mean(dim='Event')

# Calculate the change in average precipitation type accumulation per event
event_snowfall_total_average_change = E_snowfall_event_total_average - H_snowfall_event_total_average
event_rainfall_total_average_change = E_rainfall_event_total_average - H_rainfall_event_total_average
event_ice_total_average_change = E_ice_event_total_average - H_ice_event_total_average
event_fzra_total_average_change = E_fzra_event_total_average - H_fzra_event_total_average

In [15]:
### Get the duration data ready to be plotted as histograms
# Extract all the event durations pandas series
hist_snowfall_event_durations = H_snowfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_snowfall_event_durations = E_snowfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
hist_rainfall_event_durations = H_rainfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_rainfall_event_durations = E_rainfall_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
hist_ice_event_durations = H_ice_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_ice_event_durations = E_ice_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
hist_fzra_event_durations = H_fzra_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()
end_fzra_event_durations = E_fzra_5000_spatially_contiguous_event_dataset['Event_duration'].to_pandas()

# Total number of classified snowfall events
unique_snowfall_h_events = len(H_snowfall_5000_spatially_contiguous_event_dataset['Event'])
unique_snowfall_e_events = len(E_snowfall_5000_spatially_contiguous_event_dataset['Event'])
unique_rainfall_h_events = len(H_rainfall_5000_spatially_contiguous_event_dataset['Event'])
unique_rainfall_e_events = len(E_rainfall_5000_spatially_contiguous_event_dataset['Event'])
unique_ice_h_events = len(H_ice_5000_spatially_contiguous_event_dataset['Event'])
unique_ice_e_events = len(E_ice_5000_spatially_contiguous_event_dataset['Event'])
unique_fzra_h_events = len(H_fzra_5000_spatially_contiguous_event_dataset['Event'])
unique_fzra_e_events = len(E_fzra_5000_spatially_contiguous_event_dataset['Event'])

# Get unique event durations and their counts
hist_snowfall_counts = hist_snowfall_event_durations.value_counts().sort_index()
end_snowfall_counts = end_snowfall_event_durations.value_counts().sort_index()
hist_rainfall_counts = hist_rainfall_event_durations.value_counts().sort_index()
end_rainfall_counts = end_rainfall_event_durations.value_counts().sort_index()
hist_ice_counts = hist_ice_event_durations.value_counts().sort_index()
end_ice_counts = end_ice_event_durations.value_counts().sort_index()
hist_fzra_counts = hist_fzra_event_durations.value_counts().sort_index()
end_fzra_counts = end_fzra_event_durations.value_counts().sort_index()

# Reindex to fill missing durations with zero counts
hist_snowfall_counts = hist_snowfall_counts.reindex(hist_snowfall_event_durations, fill_value=0)
end_snowfall_counts = end_snowfall_counts.reindex(end_snowfall_event_durations, fill_value=0)
hist_rainfall_counts = hist_rainfall_counts.reindex(hist_rainfall_event_durations, fill_value=0)
end_rainfall_counts = end_rainfall_counts.reindex(end_rainfall_event_durations, fill_value=0)
hist_ice_counts = hist_ice_counts.reindex(hist_ice_event_durations, fill_value=0)
end_ice_counts = end_ice_counts.reindex(end_ice_event_durations, fill_value=0)
hist_fzra_counts = hist_fzra_counts.reindex(hist_fzra_event_durations, fill_value=0)
end_fzra_counts = end_fzra_counts.reindex(end_fzra_event_durations, fill_value=0)

# Calculate the 90th percentile durations for each period
snowfall_perc90_hist = np.percentile(hist_snowfall_event_durations, 90)
snowfall_perc90_end = np.percentile(end_snowfall_event_durations, 90)
rainfall_perc90_hist = np.percentile(hist_rainfall_event_durations, 90)
rainfall_perc90_end = np.percentile(end_rainfall_event_durations, 90)
ice_perc90_hist = np.percentile(hist_ice_event_durations, 90)
ice_perc90_end = np.percentile(end_ice_event_durations, 90)
fzra_perc90_hist = np.percentile(hist_fzra_event_durations, 90)
fzra_perc90_end = np.percentile(end_fzra_event_durations, 90)

In [16]:
### Print out the 90th percentile distributions
print(snowfall_perc90_hist)
print(snowfall_perc90_end)
print(rainfall_perc90_hist)
print(rainfall_perc90_end)
print(ice_perc90_hist)
print(ice_perc90_end)
print(fzra_perc90_hist)
print(fzra_perc90_end)

48.0
48.0
42.0
36.0
30.0
24.0
30.0
30.0


In [20]:
### Create an 8 panel plot of the changes in the average classified event total preciptiation and event duration distributions for 
### each precipitation type during dual frozen precipitation 5000 grid point times
# Define ticks and bar width
tick_durations = np.arange(0, 241, 24)
bar_width = 6

# Define subplot grid
fig = plt.figure(figsize=(12, 14))
gs = fig.add_gridspec(4, 2, hspace=0.3, wspace=0.2)

projection = ccrs.PlateCarree()

# Define the axes
ax1 = fig.add_subplot(gs[0, 0], projection=projection)  # Snowfall event average changes
ax2 = fig.add_subplot(gs[0, 1])                         # Snowfall event duration changes
ax3 = fig.add_subplot(gs[1, 0], projection=projection)  # Rainfall event average changes
ax4 = fig.add_subplot(gs[1, 1])                         # Rainfall event duration changes
ax5 = fig.add_subplot(gs[2, 0], projection=projection)  # Ice event average changes
ax6 = fig.add_subplot(gs[2, 1])                         # Ice event duration changes
ax7 = fig.add_subplot(gs[3, 0], projection=projection)  # Freezing rain event average changes
ax8 = fig.add_subplot(gs[3, 1])                         # Freezing rain event duration changes

######## Snowfall average event total change
# Add features
ax1.add_feature(cfeature.COASTLINE)
ax1.add_feature(cfeature.BORDERS)
ax1.add_feature(cfeature.STATES)
gl = ax1.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total snowfall change
snowfall_total_average_change_spatial = ax1.pcolormesh(event_snowfall_total_average_change['lon'], event_snowfall_total_average_change['lat'], 
                                            event_snowfall_total_average_change, transform=projection, cmap='coolwarm', vmin=-30, vmax=30)

fig.colorbar(snowfall_total_average_change_spatial, ax=ax1, label='Change in Average (mm)', shrink=0.8)
ax1.set_title('Change in Average Event Total Snowfall', fontsize=10)

######## Rain average event total change
# Add features
ax3.add_feature(cfeature.COASTLINE)
ax3.add_feature(cfeature.BORDERS)
ax3.add_feature(cfeature.STATES)
gl = ax3.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total rain change
rain_total_average_change_spatial = ax3.pcolormesh(event_rainfall_total_average_change['lon'], event_rainfall_total_average_change['lat'], 
                                            event_rainfall_total_average_change, transform=projection, cmap='coolwarm', vmin=-5, vmax=5)

fig.colorbar(rain_total_average_change_spatial, ax=ax3, label='Change in Average (mm)', shrink=0.8)
ax3.set_title('Change in Average Event Total Rainfall', fontsize=10)

######## Ice average event total change
# Add features
ax5.add_feature(cfeature.COASTLINE)
ax5.add_feature(cfeature.BORDERS)
ax5.add_feature(cfeature.STATES)
gl = ax5.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total ice change
ice_total_average_change_spatial = ax5.pcolormesh(event_ice_total_average_change['lon'], event_ice_total_average_change['lat'], 
                                            event_ice_total_average_change, transform=projection, cmap='coolwarm', vmin=-1, vmax=1)

fig.colorbar(ice_total_average_change_spatial, ax=ax5, label='Change in Average (mm)', shrink=0.8)
ax5.set_title('Change in Average Event Total Ice', fontsize=10)

######## Freezing Rain average event total change
# Add features
ax7.add_feature(cfeature.COASTLINE)
ax7.add_feature(cfeature.BORDERS)
ax7.add_feature(cfeature.STATES)
gl = ax7.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5, draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.x_inline = False
gl.rotate_labels = False

# Plot the average event total freezing rain change
fzra_total_average_change_spatial = ax7.pcolormesh(event_fzra_total_average_change['lon'], event_fzra_total_average_change['lat'], 
                                            event_fzra_total_average_change, transform=projection, cmap='coolwarm', vmin=-1, vmax=1)

fig.colorbar(fzra_total_average_change_spatial, ax=ax7, label='Change in Average (mm)', shrink=0.8)
ax7.set_title('Change in Average Event Total Freezing Rain', fontsize=10)

######## Snowfall event duration distribution changes
# Plot the duration distributions
ax2.bar(hist_snowfall_counts.index, hist_snowfall_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_snowfall_h_events})')
ax2.bar(end_snowfall_counts.index, end_snowfall_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_snowfall_e_events})')

# Add vertical lines for the 90th percentile duration
ax2.axvline(snowfall_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax2.axvline(snowfall_perc90_end, color='red', linestyle='--', linewidth=2)

ax2.set_xticks(tick_durations)
ax2.set_xticklabels(tick_durations)
ax2.set_xlim(0,240)
ax2.set_xlabel('Hours')
ax2.set_ylabel('# of Events')
ax2.set_yscale('log')
ax2.set_ylim(0.9,300)
ax2.set_title('Distribution of Snowfall Event Durations', fontsize=10)
ax2.legend(fontsize=10)

# Perform a KS test to see if the snowfall duration distributions are significantly different
ks_snow_stat, ks_snow_pvalue = ks_2samp(hist_snowfall_event_durations, end_snowfall_event_durations)

significance = ''
# Significant at the 99% confidence level
if ks_snow_pvalue < 0.01:
    significance = '**'
# Significant at the 95% confidence level
elif ks_snow_pvalue < 0.05:
    significance = '*'

# Add the KS test p-value to the bottom right corner of the subplot
ax2.text(0.98, 0.03, f'p = {ks_snow_pvalue:.3f}{significance}', transform=ax2.transAxes, fontsize=10, ha='right', va='bottom', bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

######## Rainfall event duration distribution changes
# Plot the duration distributions
ax4.bar(hist_rainfall_counts.index, hist_rainfall_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_rainfall_h_events})')
ax4.bar(end_rainfall_counts.index, end_rainfall_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_rainfall_e_events})')

# Add vertical lines for the 90th percentile duration
ax4.axvline(rainfall_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax4.axvline(rainfall_perc90_end, color='red', linestyle='--', linewidth=2)

ax4.set_xticks(tick_durations)
ax4.set_xticklabels(tick_durations)
ax4.set_xlim(0,240)
ax4.set_xlabel('Hours')
ax4.set_ylabel('# of Events')
ax4.set_yscale('log')
ax4.set_ylim(0.9,300)
ax4.set_title('Distribution of Rainfall Event Durations', fontsize=10)
ax4.legend(fontsize=10)

# Perform a KS test to see if the rainfall duration distributions are significantly different
ks_rain_stat, ks_rain_pvalue = ks_2samp(hist_rainfall_event_durations, end_rainfall_event_durations)

significance = ''
# Significant at the 99% confidence level
if ks_rain_pvalue < 0.01:
    significance = '**'
# Significant at the 95% confidence level
elif ks_rain_pvalue < 0.05:
    significance = '*'

# Add the KS test p-value to the bottom right corner of the subplot
ax4.text(0.98, 0.03, f'p = {ks_rain_pvalue:.3f}{significance}', transform=ax4.transAxes, fontsize=10, ha='right', va='bottom', bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

######## Ice event duration distribution changes
# Plot the duration distributions
ax6.bar(hist_ice_counts.index, hist_ice_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_ice_h_events})')
ax6.bar(end_ice_counts.index, end_ice_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_ice_e_events})')

# Add vertical lines for the 90th percentile duration
ax6.axvline(ice_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax6.axvline(ice_perc90_end, color='red', linestyle='--', linewidth=2)

ax6.set_xticks(tick_durations)
ax6.set_xticklabels(tick_durations)
ax6.set_xlim(0,240)
ax6.set_xlabel('Hours')
ax6.set_ylabel('# of Events')
ax6.set_yscale('log')
ax6.set_ylim(0.9,300)
ax6.set_title('Distribution of Ice Event Durations', fontsize=10)
ax6.legend(fontsize=10)

# Perform a KS test to see if the ice duration distributions are significantly different
ks_ice_stat, ks_ice_pvalue = ks_2samp(hist_ice_event_durations, end_ice_event_durations)

significance = ''
# Significant at the 99% confidence level
if ks_ice_pvalue < 0.01:
    significance = '**'
# Significant at the 95% confidence level
elif ks_ice_pvalue < 0.05:
    significance = '*'

# Add the KS test p-value to the bottom right corner of the subplot
ax6.text(0.98, 0.03, f'p = {ks_ice_pvalue:.3f}{significance}', transform=ax6.transAxes, fontsize=10, ha='right', va='bottom', bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

######## Freezing rain event duration distribution changes
# Plot the duration distributions
ax8.bar(hist_fzra_counts.index, hist_fzra_counts.values, width=bar_width, color='blue', edgecolor='black', alpha=0.7, label=f'Historical (N = {unique_fzra_h_events})')
ax8.bar(end_fzra_counts.index, end_fzra_counts.values, width=bar_width, color='red', edgecolor='black', alpha=0.7, label=f'End of Century (N = {unique_fzra_e_events})')

# Add vertical lines for the 90th percentile duration
ax8.axvline(fzra_perc90_hist, color='blue', linestyle='--', linewidth=2)
ax8.axvline(fzra_perc90_end, color='red', linestyle='--', linewidth=2)

ax8.set_xticks(tick_durations)
ax8.set_xticklabels(tick_durations)
ax8.set_xlim(0,240)
ax8.set_xlabel('Hours')
ax8.set_ylabel('# of Events')
ax8.set_yscale('log')
ax8.set_ylim(0.9,300)
ax8.set_title('Distribution of Freezing Rain Event Durations', fontsize=10)
ax8.legend(fontsize=10)

# Perform a KS test to see if the freezing rain duration distributions are significantly different
ks_fzra_stat, ks_fzra_pvalue = ks_2samp(hist_fzra_event_durations, end_fzra_event_durations)

significance = ''
# Significant at the 99% confidence level
if ks_fzra_pvalue < 0.01:
    significance = '**'
# Significant at the 95% confidence level
elif ks_fzra_pvalue < 0.05:
    significance = '*'

# Add the KS test p-value to the bottom right corner of the subplot
ax8.text(0.98, 0.03, f'p = {ks_fzra_pvalue:.3f}{significance}', transform=ax8.transAxes, fontsize=10, ha='right', va='bottom', bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

# Add panel labels
ax1.text(0.04, 0.98, '(a)', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax2.text(0.04, 0.98, '(b)', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax3.text(0.04, 0.98, '(c)', transform=ax3.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax4.text(0.04, 0.98, '(d)', transform=ax4.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax5.text(0.04, 0.98, '(e)', transform=ax5.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax6.text(0.04, 0.98, '(f)', transform=ax6.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax7.text(0.04, 0.98, '(g)', transform=ax7.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')
ax8.text(0.04, 0.98, '(h)', transform=ax8.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

plt.suptitle('Changes in Average Classified Event Total Precipitation Type and Duration Distributions During Times With\nAt Least Two Types of Frozen Precipitation Spanning 5000 Grid Points\nEnd of Century - Historical')

plt.savefig('Key Figures/Changes in Average Classified Event Total Precipitation and Duration Distributions During 5000 Grid Point Dual Precipitation Times',bbox_inches = 'tight')

plt.close()